In [1]:
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'project-dog/dogImages-resized'

from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
print (training_image)

arn:aws:iam::319963865301:role/service-role/AmazonSageMaker-ExecutionRole-20200503T163120
825641698319.dkr.ecr.us-east-2.amazonaws.com/image-classification:latest
CPU times: user 730 ms, sys: 83.8 ms, total: 814 ms
Wall time: 924 ms


In [3]:
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3valid = 's3://{}/{}/valid/'.format(bucket, prefix)
s3test = 's3://{}/{}/test/'.format(bucket, prefix)
print(s3train)
print(s3valid)
print(s3test)

s3://sagemaker-us-east-2-319963865301/project-dog/dogImages-resized/train/
s3://sagemaker-us-east-2-319963865301/project-dog/dogImages-resized/valid/
s3://sagemaker-us-east-2-319963865301/project-dog/dogImages-resized/test/


In [11]:
train = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation = sagemaker.session.s3_input(s3valid, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')
train_lst = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_lst = sagemaker.session.s3_input(s3valid, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')

data_channels = {'train': train, 'validation': validation, 'train_lst': train_lst, 'validation_lst': validation_lst}

In [21]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
ic = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p2.xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess,
                                         base_job_name='project-dog',
                                  )
ic.set_hyperparameters(num_layers=18,
                             use_pretrained_model=1,
                             image_shape = "3,224,224",
                             num_classes=133,
                             num_training_samples=6678,
                             mini_batch_size=32,
                             epochs=30,
                             learning_rate=0.001,
                             precision_dtype='float32',
                             early_stopping=True,
                             optimizer='adam',
                             checkpoint_frequency=5,
                    )

In [22]:
ic.fit(data_channels, wait=False)